In [1]:
# standard libraries import
import os
import glob
import json
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Tuple, Any

import cv2
import matplotlib.pyplot as plt


## reading as df-like format using geopandas library
import geopandas as gpd

## transformation to lat, long coordinates
import pyproj
import utm

In [2]:
rootDir = Path('metadata')
floor_map = {"B2": -2, "B1": -1, "F1": 0, "F2": 1, "F3": 2, "F4": 3, "F5": 4, "F6": 5, "F7": 6, "F8": 7, "F9": 8,
             "1F": 0, "2F": 1, "3F": 2, "4F": 3, "5F": 4, "6F": 5, "7F": 6, "8F": 7, "9F": 8}
sampleCsvPath = 'sample_submission.csv'
imageOutputDir = 'referencePublicNotebooks/floorDataImages'

proj = pyproj.Transformer.from_crs(3857, 4326, always_xy=True)
maxFloorDimension = 440.0
maxFigureSize = 22.0

In [3]:
def generate_target_buildings() -> List[str]:
    ssubm = pd.read_csv(sampleCsvPath)
    ssubm_df = ssubm["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
    buildingsList = sorted(ssubm_df[0].value_counts().index.tolist()) # type: ignore
    return buildingsList

In [4]:
def get2DPolygonPoints(arr):
    polygonPoints = None
    numElements = arr.size
    if((numElements > 0) and (numElements % 2 == 0)):
        polygonPoints = arr.reshape(-1,2)
    return polygonPoints

def getXYCoordinates(polygonData):
    x, y = polygonData.exterior.coords.xy
    x, y = np.array(x), np.array(y)
    longLat = np.column_stack(proj.transform(x, y))
    easting, northing, _, _ = utm.from_latlon(longLat[:,1], longLat[:,0])
    utmCoordinates = np.column_stack((easting, northing))
    return get2DPolygonPoints(utmCoordinates)
    
def getReferenceUTMPoint(geoJsonDf):
    referenceUTMPoint = np.array([0.0, 0.0])
    boundingPolygon = geoJsonDf.loc[0, 'utm']
    if((boundingPolygon is not None) and (boundingPolygon.size %2 == 0)):
        minx = np.min(boundingPolygon[:,0])
        miny = np.min(boundingPolygon[:,1])
        referenceUTMPoint = np.array([minx, miny])
    return referenceUTMPoint

def utmToLocal(origPoint, refPoint):
    return origPoint - refPoint

In [5]:
def getGeoJsonInfo(jsonFilePath):
    # Opening JSON file 
    f = open(jsonFilePath)
    # returns JSON object as  a dictionary 
    geoJsonData = json.load(f) 

    boundaryPoints = []
    for index, feature in enumerate(geoJsonData['features']):
        ## espgs -> latlong -> utm -> local coordinates
        pts = np.array(feature['geometry']['coordinates'])
        try:    
            if len(pts.shape) <= 2:
                pts = pts.reshape(-1,2)
            else:
                pts = np.squeeze(pts, axis=0)
            
            longLat = np.column_stack(proj.transform(pts[:,0], pts[:,1]))
            easting, northing, _, _ = utm.from_latlon(longLat[:,1], longLat[:,0])
            utmCoordinates = np.column_stack((easting, northing))
            if index == 0:
                minx = np.min(utmCoordinates[:,0])
                miny = np.min(utmCoordinates[:,1])
                leftMostPoint = np.array([minx, miny])
            boundaryPoints.append(utmCoordinates - leftMostPoint)
        except:
            pass
            ## print(pts.shape)
            ##print(feature['geometry']['coordinates'])
    coveredPercentage = (float(len(boundaryPoints)) / float(len(geoJsonData['features']))) * 100.0
    ##print(coveredPercentage)
    return boundaryPoints

In [6]:
def getFloorInfoJson(jsonFilePath):
    # Opening JSON file 
    f = open(jsonFilePath)
    # returns JSON object as  a dictionary 
    floorInfo = json.load(f) 
    return np.array([floorInfo['map_info']['width'],floorInfo['map_info']['height']])

In [7]:
def getFigureSize(floorInfo):
    figureSize = np.round((floorInfo/maxFloorDimension)  * maxFigureSize)
    return figureSize

In [8]:
buildingsList = generate_target_buildings()

```python
for index,building in enumerate(buildingsList):
    ## building = buildingsList[0]
    building_path = rootDir / building
    folders = sorted(building_path.glob('*'))
    print(f"{index} bdg - {building}")

    for i,folder in enumerate(folders):
        ##print(folder.name)
        ## geoJson boudnary points
        geoJsonPath = sorted(folder.glob("*_map.json"))[0]
        break
        
        floorBoundaryPoints = getGeoJsonInfo(geoJsonPath)
        
        ## scale of image
        floorDataJsonPath = sorted(folder.glob("*_info.json"))[0]
        floorInfo = getFloorInfoJson(floorDataJsonPath)
        figureSize = getFigureSize(floorInfo)
        
        plt.ioff()
        plt.figure(figsize=(figureSize[0],figureSize[1]), frameon=False)
        for polygon in floorBoundaryPoints:
            plt.plot(polygon[:,0], polygon[:,1], 'k')
        plt.axis('off')
        plt.savefig(f"{imageOutputDir}/{index}_{building}_{folder.name}.png", dpi=300, bbox_inches='tight', pad_inches=0);
        plt.close()
    break
```        

In [ ]:
##for index,building in enumerate(buildingsList):    
##    if index > 3:
index = 0 
building = buildingsList[index]
building_path = rootDir / building
folders = sorted(building_path.glob('*'))
print(f"{index} bdg - {building}")

for i,folder in enumerate(folders):
    ## geoJson boundary points
    geoJsonPath = sorted(folder.glob("*_map.json"))[0]
    geoJsonData = gpd.read_file(geoJsonPath).loc[:,['id','geometry']]
    print(f"geoJsonData shape is {geoJsonData.shape}")

    ## scale of image
    floorDataJsonPath = sorted(folder.glob("*_info.json"))[0]
    floorInfo = getFloorInfoJson(floorDataJsonPath)
    figureSize = getFigureSize(floorInfo)

    geoJsonData['utm'] = geoJsonData.apply(lambda row : getXYCoordinates(row['geometry']), axis=1)
    referenceUTMPoint = getReferenceUTMPoint(geoJsonData)
    geoJsonData['localPts'] = geoJsonData.apply(lambda row : utmToLocal(row['utm'], referenceUTMPoint), axis=1)
    floorBoundaryPoints = geoJsonData['localPts'].values.tolist()

    plt.ioff()
    plt.figure(figsize=(figureSize[0],figureSize[1]), frameon=False)
    for polygon in floorBoundaryPoints:
        if polygon is not None:
            plt.plot(polygon[:,0], polygon[:,1], 'k')
    plt.axis('off')
    plt.savefig(f"{imageOutputDir}/{index}_{building}_{folder.name}.png", dpi=300, bbox_inches='tight', pad_inches=0);
    plt.close()
## break

img = np.expand_dims(cv2.imread('5a0546857ecc773753327266_B1.png', cv2.IMREAD_GRAYSCALE), axis=0)
temp = np.repeat(img,3,axis=0)
print(temp.dtype, temp.shape)